In [1]:
# Chosen using results from Bipolar-K
epsilon = 0.75
k = 250

input_dim = 13166
num_clusters = 18

model_file = "Model/model/perplexity_10_regularizer_0.001_batch_size_512_learning_rate_0.01_latent_dimension_2_activation_ELU_seed_1_iter_5400.ckpt"
pc_matrix_file = "Data/bipolar_pc.tsv"

import csv
import json
import matplotlib
import numpy as np
import os
import pandas as pd

os.system("rm *.png")

from load_vae import load_vae

import sys
sys.path.insert(0, "../Code/")
from explain_cs import explain
from metrics import metrics, eval_epsilon
from myplot import plot_polys, plot_groups, plot_metrics, plot_explanation
from misc import poly2labels
from train_ae import train_ae
matplotlib.rc("lines", markersize = 4)# Override some default plot parameters

sys.path.insert(0, "../Bipolar/")
from load_MLP import load_model


In [2]:
# Load Data & Configuration

x = pd.read_csv("Data/bipolar_rep_scaled.tsv", sep="\t").values

with open("vertices.json") as json_file:  
    all_vertices = json.load(json_file)


In [3]:
# Approximate the original data

matrix = pd.read_csv(pc_matrix_file, sep = "\t", header = None).values

x_full = np.matmul(x, np.transpose(matrix))

In [ ]:
# Calculate Data Representation
# train_ae(x_full,
#           encoder_shape = [100, 100, 100, 2], decoder_shape = [2, 100, 100, 100],
#           learning_rate = 0.001, batch_size = 4, min_epochs = 100, stopping_epochs =50 , tol = 0.001, freq_eval = 1,
#           device=None)
model = load_model('Model/model_encoder.pt')

x_full = torch.from_numpy(x_full).float()
data_rep = model(x_full)

x_full = x_full.detach().numpy()
data_rep = data_rep.detach().numpy().squeeze()

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
INFO:tensorflow:Restoring parameters from Model/model/perplexity_10_regularizer_0.001_batch_size_512_learning_rate_0.01_latent_dimension_2_activation_ELU_seed_1_iter_5400.ckpt


In [ ]:
# Visualize the data and the marked clusters
plot_polys(data_rep, all_vertices)

In [ ]:
# Visualize the clusters in the learned representation

labels = poly2labels(data_rep, all_vertices)

means, centers, indices = plot_groups(x_full, data_rep, num_clusters, labels, name = "bipolar-rep.png")


In [ ]:
# Evaluate our explanations

deltas = np.load("../Bipolar-K/deltas" + str(k) + ".npy")

a, b = metrics(load_model, x_full, indices, deltas, epsilon, k = k)

plot_metrics(a, b, "bipolar-metrics.png")

plot_explanation(load_model, x_full, data_rep, indices, deltas, a, b, 4, 5, k = k, name = "bipolar-close.png", num_points = 200)

plot_explanation(load_model, x_full, data_rep, indices, deltas, a, b, 3, 17, k = k, name = "bipolar-far.png", num_points = 200)

plot_explanation(load_model, x_full, data_rep, indices, deltas, a, b, 1, 9, k = k, name = "bipolar-hard.png", num_points = 200)


In [ ]:
# Evaluate Truncated DBM 

exp_mean = np.zeros((num_clusters - 1, input_dim))
for i in range(num_clusters - 1):
    exp_mean[i, :] = means[i + 1] - means[0]
    
a, b = metrics(load_model, x_full, indices, exp_mean, epsilon, k = k)

plot_metrics(a, b, name = "bipolar-metrics-means.png")

plot_explanation(load_model, x_full, data_rep, indices, exp_mean, a, b, 3, 17, k = k, name = "bipolar-far-means-k.png", num_points = 200)



In [ ]:
# Evaluate DBM
a, b = metrics(load_model, x_full, indices, exp_mean, epsilon)

plot_explanation(load_model, x_full, data_rep, indices, exp_mean, a, b, 3, 17, name = "bipolar-far-means.png", num_points = 200)
